# Koiter shells with kinks
The HHJ bending moment is normal-normal continuous, $\sigma_{\hat \mu\hat\mu}$, which means that the bending moments are preserved over elements. Even if the structure has a non-smooth kink. What flows in from one element exactly flows out to the other element. Therefore the moment balance equation is fulfilled.

<center>
<img src="kink_struct.png" width="150"> 
</center>

In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.meshes import MakeStructuredSurfaceMesh
from ngsolve.webgui import Draw

Put the whole method in a function.

In [ ]:
def MaterialNorm(mat, E, nu):
    return E/(1-nu**2)*((1-nu)*InnerProduct(mat,mat)+nu*Trace(mat)**2)
def MaterialNormInv(mat, E, nu):
    return (1+nu)/E*(InnerProduct(mat,mat)-nu/(2*nu+1)*Trace(mat)**2)

def SetUp(order, fes, E, nu, thickness, mesh, force, clamped_bnd):
    u,sigma,hyb = fes.TrialFunction()
    # trace needed as we are on the surface
    sigma, hyb = sigma.Trace(), hyb.Trace()
    nsurf = specialcf.normal(mesh.dim)
    t     = specialcf.tangential(mesh.dim)
    nel   = Cross(nsurf, t)
    
    Ptau    = Id(mesh.dim) - OuterProduct(nsurf,nsurf)
    Ftau    = grad(u).Trace() + Ptau
    Ctau    = Ftau.trans*Ftau
    Etautau = 0.5*(Ctau - Ptau)
    
    nphys   = Normalize(Cof(Ftau)*nsurf)
    tphys   = Normalize(Ftau*t)
    nelphys = Cross(nphys,tphys)
    gradn = specialcf.Weingarten(3)
    Hn = CF( (u.Operator("hesseboundary").trans*nphys), dims=(3,3) )
    
    fesVF = VectorFacetSurface(mesh, order=order-1)
            
    gfclamped = GridFunction(FacetSurface(mesh,order=0))
    gfclamped.Set(1, definedon=mesh.BBoundaries(clamped_bnd))
    
    solution = GridFunction(fes, name="solution")
    averednv = GridFunction(fesVF)
    averednv_start = GridFunction(fesVF)
    
    cfnphys = Normalize(Cof(Ptau+grad(solution.components[0]))*nsurf)
    
    cfn  = Normalize(CoefficientFunction( averednv.components ))
    cfnR = Normalize(CoefficientFunction( averednv_start.components ))
    pnaverage = Normalize( cfn - (tphys*cfn)*tphys )
    

    averednv.Set(nsurf, dual=True, definedon=mesh.Boundaries(".*"))
    averednv_start.Set(nsurf, dual=True, definedon=mesh.Boundaries(".*"))
    
    bfA = BilinearForm(fes, symmetric=True, condense=True)
    #membrane energy
    bfA += Variation( 0.5*thickness*MaterialNorm(Etautau, E, nu)*ds )
    #bending energy
    bfA += Variation( (-6/thickness**3*MaterialNormInv(sigma, E, nu) \
                       + InnerProduct(Hn + (1-nsurf*nphys)*gradn,sigma))*ds ).Compile()
    # boundary term of bending energy including hybridization variable
    bfA += Variation( (acos(nelphys*pnaverage)-acos(nel*cfnR)+hyb*nel)*sigma[nel,nel]*ds(element_boundary=True) ).Compile()
    # right-hand side
    bfA += Variation( force )
    return bfA, solution, averednv, gfclamped, cfnphys

Let's consider a kink structure, which is clamped on the right and on the top left edge we apply a shear force.
<center>
<img src="kink_shear.png" width="150"> 
</center>
How does the bending moment distribute over the shell?

In [ ]:
order = 3

def mapping(x, y, z):
    return (20*(x-0.5) if x >= 0.5 else 0, 5*y,0 if x >= 0.5 else 20*(0.5-x) )
mesh = MakeStructuredSurfaceMesh(quads=False, nx=10, ny=4, mapping=mapping)
Draw(mesh)

force = CF( (1,0,0) )
        
E, nu = 1e6, 0
thickness = 0.1

fes1 = HDivDivSurface(mesh, order=order-1, discontinuous=True)
fes2 = VectorH1(mesh, order=order, dirichlet_bbnd="right")
fes3 = NormalFacetSurface(mesh, order=order-1, dirichlet_bbnd="right")
fes  = fes2*fes1*fes3
u,sigma,hyb = fes.TrialFunction()
par = Parameter(0.0)

bfA, solution, averednv, gfclamped, cfnphys = SetUp(order, fes, E, nu, thickness, mesh, \
                                            -par*force*u*dx(definedon=mesh.BBoundaries("left")), clamped_bnd="right")

In [ ]:
solution.vec[:]=0
scene = Draw(Norm(solution.components[1]), mesh, "disp", deformation=solution.components[0])

In [ ]:
numsteps=10
rf = averednv.vec.CreateVector()
with TaskManager():
    for steps in range(numsteps):
        par.Set((steps+1)/numsteps)
        print("Loadstep =", steps+1, ", F/Fmax =", (steps+1)/numsteps*100, "%")
        
        # update averaged normal vector
        averednv.Set(
            (1 - gfclamped) * cfnphys + gfclamped * specialcf.normal(3),
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )
        
        # solve
        solvers.Newton(bfA, solution, inverse="sparsecholesky", printing=False, maxerr=2e-10, maxit=20)
        scene.Redraw()

Note, that the angle of the kink gets preserved during deformation.

Next consider a kink structure, which is again clamped on the right, but now we apply a bending moment on the top left edge.
<center>
<img src="kink_bend.png" width="150"> 
</center>
How does the bending moment look like?

In [ ]:
order = 3

def mapping(x, y, z):
    return (2*(x-0.5) if x >= 0.5 else 0, y,0 if x >= 0.5 else 2*(0.5-x) )
mesh = MakeStructuredSurfaceMesh(quads=False, nx=10, ny=4, mapping=mapping)
Draw(mesh)

force = IfPos(z-1+1e-6, 1, 0)*Cross(specialcf.normal(3),specialcf.tangential(3))
        
E, nu = 8e6, 0
thickness = 0.01

fes1 = HDivDivSurface(mesh, order=order-1, discontinuous=True)
fes2 = VectorH1(mesh, order=order, dirichlet_bbnd="right")
fes3 = NormalFacetSurface(mesh, order=order-1, dirichlet_bbnd="right")
fes  = fes2*fes1*fes3
u,sigma,hyb = fes.TrialFunction()
par = Parameter(0.0)

bfA, solution, averednv, gfclamped, cfnphys = SetUp(order, fes, E, nu, thickness, mesh, \
                                           -par*force*hyb.Trace()*ds(element_boundary=True), clamped_bnd="right")

In [ ]:
solution.vec[:]=0
scene = Draw(Norm(solution.components[1]), mesh, "disp", deformation=solution.components[0])

In [ ]:
numsteps=10
rf = averednv.vec.CreateVector()
with TaskManager():
    for steps in range(numsteps):
        par.Set((steps+1)/numsteps)
        print("Loadstep =", steps+1, ", F/Fmax =", (steps+1)/numsteps*100, "%")
        
        # update averaged normal vector
        averednv.Set(
            (1 - gfclamped) * cfnphys + gfclamped * specialcf.normal(3),
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )
        
        # solve
        solvers.Newton(bfA, solution, inverse="sparsecholesky", printing=False, maxerr=2e-10, maxit=20)
        scene.Redraw()

We can also consider structures where some edges share more than two elements. The T-structure is such a branched shell. We clamp it at the bottom and apply a shear force on the top left edge. We again expect that a linear bending moment will be induced, but what happens to the right branch?

In [ ]:
%run tstructmesh.py

In [ ]:
mesh = MakeTStructureMesh()
Draw(mesh)

order=3

thickness = 0.1
E, nu = 6.1e6, 0

force = CF( (0,0,1e3) )

fes1 = HDivDivSurface(mesh, order=order-1, discontinuous=True)
fes2 = VectorH1(mesh, order=order, dirichlet_bbnd="downbottom")
fes3 = NormalFacetSurface(mesh, order=order-1, dirichlet_bbnd="downbottom")
fes  = fes2*fes1*fes3
u,sigma,hyb = fes.TrialFunction()
par = Parameter(0.0)

bfA, solution, averednv, gfclamped, cfnphys = SetUp(order, fes, E, nu, thickness, mesh, \
                                            -par*force*u*dx(definedon=mesh.BBoundaries("upbottom")), clamped_bnd="downbottom")

In [ ]:
solution.vec[:]=0
scene = Draw(Norm(solution.components[1]), mesh, "disp", deformation=solution.components[0])

In [ ]:
numsteps=10
rf = averednv.vec.CreateVector()
with TaskManager():
    for steps in range(numsteps):
        par.Set((steps+1)/numsteps)
        print("Loadstep =", steps+1, ", F/Fmax =", (steps+1)/numsteps*100, "%")
        
        # update averaged normal vector
        averednv.Set(
            (1 - gfclamped) * cfnphys + gfclamped * specialcf.normal(3),
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )
        
        # solve
        solvers.Newton(bfA, solution, inverse="sparsecholesky", printing=False, maxerr=2e-10, maxit=20)
        scene.Redraw()